Calibrating all three camera simultaneously

In [1]:
import pyrealsense2 as rs
import cv2
import numpy as np

def calibrate_camera(pipeline, num_images, pattern_size):
    # Initialize arrays to store object points and image points
    objpoints = []  # 3D points in real-world coordinates
    imgpoints = []  # 2D points in image coordinates
    
    # Prepare the object points (assuming a flat calibration pattern)
    objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

    # Capture images for calibration
    image_count = 0
    while image_count < num_images:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()

        if not color_frame:
            continue

        # Convert RealSense frame to numpy array
        frame = np.asanyarray(color_frame.get_data())

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret_corners, corners = cv2.findChessboardCorners(gray, pattern_size, None)

        # If corners are found, add them to the calibration data
        if ret_corners:
            objpoints.append(objp)
            corners_refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                                (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            imgpoints.append(corners_refined)

            # Draw and display the corners on the image
            frame_with_corners = cv2.drawChessboardCorners(frame, pattern_size, corners_refined, ret_corners)
            cv2.imshow('Calibration Capture', frame_with_corners)

            # Save the image (optional)
            cv2.imwrite(f'calibration_image_{image_count + 1}.png', frame)

            image_count += 1

        # Display the frame
        cv2.imshow('Calibration Capture', frame)

        # Wait for a key press to move to the next frame
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

    return objpoints, imgpoints

# Set the number of images to capture for calibration
num_images = 20

# Set the size of the chessboard pattern (number of inner corners)
pattern_size = (7, 6)  # Adjust the size according to your calibration pattern
frameSize = (640,480)
# Create pipeline objects for each camera
pipeline1 = rs.pipeline()
pipeline2 = rs.pipeline()
pipeline3 = rs.pipeline()

# Configure pipeline for camera 1
config1 = rs.config()
config1.enable_device('136622073300')  # Replace with the serial number of your first camera
config1.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline1.start(config1)

# Configure pipeline for camera 2
config2 = rs.config()
config2.enable_device('138422074931')  # Replace with the serial number of your second camera
config2.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline2.start(config2)

# Configure pipeline for camera 3
config3 = rs.config()
config3.enable_device('141722070246')  # Replace with the serial number of your third camera
config3.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline3.start(config3)

print("Calibrating camera 1...")
objpoints1, imgpoints1 = calibrate_camera(pipeline1, num_images, pattern_size)

print("Calibrating camera 2...")
objpoints2, imgpoints2 = calibrate_camera(pipeline2, num_images, pattern_size)

print("Calibrating camera 3...")
objpoints3, imgpoints3 = calibrate_camera(pipeline3, num_images, pattern_size)

# Release the cameras and close all OpenCV windows
pipeline1.stop()
pipeline2.stop()
pipeline3.stop()
cv2.destroyAllWindows()

# Perform camera calibration only if there are valid calibration images
if len(objpoints1) > 0 and len(objpoints2) > 0 and len(objpoints3) > 0:
    # Perform camera calibration for camera 1
    ret1, mtx1, dist1, rvecs1, tvecs1 = cv2.calibrateCamera(objpoints1, imgpoints1, frameSize, None, None)

    # Save the calibration parameters for camera 1 (optional)
    np.savez('camera1_calibration.npz', ret=ret1, mtx=mtx1, dist=dist1, rvecs=rvecs1, tvecs=tvecs1)
    print("Camera 1 calibration complete. Calibration parameters saved.")

    # Perform camera calibration for camera 2
    ret2, mtx2, dist2, rvecs2, tvecs2 = cv2.calibrateCamera(objpoints2, imgpoints2, frameSize, None, None)

    # Save the calibration parameters for camera 2 (optional)
    np.savez('camera2_calibration.npz', ret=ret2, mtx=mtx2, dist=dist2, rvecs=rvecs2, tvecs=tvecs2)
    print("Camera 2 calibration complete. Calibration parameters saved.")

    # Perform camera calibration for camera 3
    ret3, mtx3, dist3, rvecs3, tvecs3 = cv2.calibrateCamera(objpoints3, imgpoints3, frameSize, None, None)

    # Save the calibration parameters for camera 3 (optional)
    np.savez('camera3_calibration.npz', ret=ret3, mtx=mtx3, dist=dist3, rvecs=rvecs3, tvecs=tvecs3)
    print("Camera 3 calibration complete. Calibration parameters saved.")
else:
    print("No valid calibration images captured for one or more cameras.")


Calibrating camera 1...
Calibrating camera 2...
Calibrating camera 3...
Camera 1 calibration complete. Calibration parameters saved.
Camera 2 calibration complete. Calibration parameters saved.
Camera 3 calibration complete. Calibration parameters saved.


In [6]:
# Print calibration parameters for camera 1
print("Camera 1 Calibration Parameters:")
print(" - Ret:", ret1)
print(" - Camera Matrix:\n", mtx1)
print(" - Distortion Coefficients:\n", dist1)
print(" - Rotation Vectors:\n", rvecs1)
print(" - Translation Vectors:\n", tvecs1)

Camera 1 Calibration Parameters:
 - Ret: 0.1254097583124365
 - Camera Matrix:
 [[528.58801898   0.         312.94386755]
 [  0.         527.56178967 230.53829767]
 [  0.           0.           1.        ]]
 - Distortion Coefficients:
 [[ 1.63401603e-02  4.85356617e-01 -3.03997384e-03  9.58432952e-04
  -1.40621728e+00]]
 - Rotation Vectors:
 (array([[-0.07190014],
       [ 0.06976468],
       [ 3.0796974 ]]), array([[ 0.02325153],
       [-0.08309231],
       [-0.04609755]]), array([[-0.14021792],
       [-0.05066379],
       [ 3.09279286]]), array([[-0.16508751],
       [-0.05464952],
       [ 3.09229415]]), array([[-0.16843463],
       [-0.06462041],
       [ 3.09422607]]), array([[-0.16946026],
       [-0.07607392],
       [ 3.09249565]]), array([[-0.17051233],
       [-0.07143511],
       [ 3.09292916]]), array([[-0.17608967],
       [-0.07505397],
       [ 3.09294497]]), array([[-0.17287209],
       [-0.06120735],
       [ 3.09341077]]), array([[ 0.02246638],
       [-0.11335026],


In [7]:
print("Camera 2 Calibration Parameters:")
print(" - Ret:", ret2)
print(" - Camera Matrix:\n", mtx2)
print(" - Distortion Coefficients:\n", dist2)
print(" - Rotation Vectors:\n", rvecs2)
print(" - Translation Vectors:\n", tvecs2)

Camera 2 Calibration Parameters:
 - Ret: 0.2910936528464632
 - Camera Matrix:
 [[577.33636586   0.         299.70133926]
 [  0.         577.83181953 226.7769114 ]
 [  0.           0.           1.        ]]
 - Distortion Coefficients:
 [[ 0.11128584  0.11810827 -0.00250568 -0.0130943  -0.54679752]]
 - Rotation Vectors:
 (array([[-0.65262696],
       [-0.12370743],
       [ 2.99903335]]), array([[ 0.2222452 ],
       [-0.0958817 ],
       [ 3.12990322]]), array([[-0.05941258],
       [ 0.18397354],
       [-1.55089368]]), array([[-0.05705871],
       [ 0.17051702],
       [-1.53978659]]), array([[-0.06273067],
       [ 0.16580396],
       [-1.53687572]]), array([[-0.0598216 ],
       [ 0.16489102],
       [-1.53587935]]), array([[-0.06298248],
       [ 0.16242056],
       [-1.53752085]]), array([[-0.06090474],
       [ 0.16223097],
       [-1.53712763]]), array([[-0.00487026],
       [ 0.17087126],
       [-1.52683689]]), array([[-0.1524355 ],
       [ 0.19280578],
       [-3.10894022]])

In [8]:
print("Camera 3 Calibration Parameters:")
print(" - Ret:", ret3)
print(" - Camera Matrix:\n", mtx3)
print(" - Distortion Coefficients:\n", dist3)
print(" - Rotation Vectors:\n", rvecs3)
print(" - Translation Vectors:\n", tvecs3)


Camera 3 Calibration Parameters:
 - Ret: 0.14377376941959086
 - Camera Matrix:
 [[771.14433512   0.         379.67988701]
 [  0.         771.55889462 213.44866422]
 [  0.           0.           1.        ]]
 - Distortion Coefficients:
 [[ 0.0732803   0.86325872 -0.00741682  0.01265033 -3.32656379]]
 - Rotation Vectors:
 (array([[-0.20428506],
       [ 0.2420131 ],
       [ 2.93239083]]), array([[-0.16657725],
       [ 0.20195182],
       [ 2.95071834]]), array([[-0.20342855],
       [ 0.19890662],
       [ 2.96815148]]), array([[-0.21340793],
       [ 0.24455112],
       [ 2.98247336]]), array([[-0.2314805 ],
       [ 0.31476122],
       [ 2.94028009]]), array([[-0.0875658 ],
       [-0.23959013],
       [-0.10822099]]), array([[-0.38507855],
       [ 0.15962402],
       [ 2.99915532]]), array([[ 0.1459837 ],
       [-0.31070859],
       [-1.67860549]]), array([[-0.48839462],
       [ 0.13950645],
       [ 2.97201446]]), array([[-0.14877965],
       [ 0.06217332],
       [ 3.00854922]]